In [1]:
import cv2
import sys
import time
import numpy as np
import jsonpickle
from message import Message
from camera import Camera
from publisher import Publisher
from keypress_listener import KeypressListener
import ev3_dc as ev3
from thread_task import Periodic, Task, STATE_STARTED, STATE_FINISHED, STATE_STOPPED, STATE_TO_STOP
from threading import Event, Thread
from threads import Repeated
from slam import EKFSLAM
import math
from main import Main
from multiprocessing.pool import ThreadPool

In [2]:
camera = Camera()
count = 0
publisher = Publisher()
camera_height = 0.285

def close(self):
        camera.close()

In [3]:
def transformImage(img):
        aruco_dict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_ARUCO_ORIGINAL)        
        parameters = cv2.aruco.DetectorParameters_create()  
        positions = []
        distances = []
        angles = []
        if img is not None:
            corners, ids, rejected_img_points = cv2.aruco.detectMarkers(img, aruco_dict, parameters=parameters)
            
            if ids is not None:
                for j in range(0, len(ids)):
                    rvec, tvec, _ = cv2.aruco.estimatePoseSingleMarkers(corners[j], 0.048, camera.camera_matrix, camera.dist_coeffs)
                    # (rvec - tvec).any()

                    hypothenuse = np.linalg.norm(tvec)
                    while True:
                        try:    
                            distance = np.sqrt(hypothenuse**2 - camera_height**2)
                            break
                        except:
                            continue
                    distances.append(distance)
                    pos_x = tvec[0][0][0]
                    pos_y = np.sqrt(distance**2 - pos_x**2)
                    angle = (np.arctan2(-pos_x,pos_y))
                    angles.append(angle)


                    positions.append([pos_x, pos_y])
                    cv2.aruco.drawDetectedMarkers(img, corners)
                    cv2.drawFrameAxes(img, camera.camera_matrix, camera.dist_coeffs, rvec, tvec, 0.048)
            else:
                ids = []
        else:
            ids = []


        return img, ids, positions, distances, angles


In [4]:
def watch():
    global count
    print("Check!")
    try:
        # get image from the camera
        _, raw_img = camera.read()
        
        # get the time
        time0 = time.time()

        # imaginary processing
        img, ids, positions, distances, angles = transformImage(raw_img)

        # curr_state = drive.state

        # create message
        msg = Message(
            id = count,
            timestamp = time0,
            start = True,
            
            landmark_ids = ids,
            landmark_rs = distances,
            landmark_alphas = angles,
            landmark_positions = positions,

            landmark_estimated_ids = [],
            landmark_estimated_positions = [],
            landmark_estimated_stdevs = [],

            robot_position = np.array([0.0, 0.0]),
            robot_theta = 0.0,
            robot_stdev = [0.5, 0.5, 0.5],

        )
        count += 1

        # pickle message
        msg_str = jsonpickle.encode(msg)

        # publish message and image
        publisher.publish_img(msg_str, img)

        # if curr_state == STATE_STOPPED:
        #     return True

        if len(distances) > 0:
            return min(distances)
    
        return np.inf

    except KeyboardInterrupt as e:
        # tidy up
        print(e)
        close()

In [5]:
vehicle = ev3.TwoWheelVehicle (
    0.0210,  # radius_wheel
    0.1350,  # tread
    protocol=ev3.USB
) 

In [6]:
stopped = False

while not stopped:
    pool = ThreadPool(processes=2)
    watch_results = pool.apply_async(watch)
    distance = watch_results.get()
    if vehicle._current_movement is None:
        pool.apply_async(vehicle.drive_straight(0.1).start)

    if distance < 0.4:
        pool.close()
        stopped = True

vehicle.stop()


Check!
1.087031937276938
ANOTHER ROUND!
Check!
1.087031937276938
ANOTHER ROUND!
Check!
1.087031937276938
ANOTHER ROUND!
Check!
1.087031937276938
ANOTHER ROUND!
Check!
inf
ANOTHER ROUND!
Check!
inf
ANOTHER ROUND!
Check!
inf
ANOTHER ROUND!
Check!
inf
ANOTHER ROUND!
Check!
inf
ANOTHER ROUND!
Check!
1.084271466275239
ANOTHER ROUND!
Check!
1.084271466275239
ANOTHER ROUND!
Check!
inf
ANOTHER ROUND!
Check!
inf
ANOTHER ROUND!
Check!
inf
ANOTHER ROUND!
Check!
inf
ANOTHER ROUND!
Check!
inf
ANOTHER ROUND!
Check!
1.0934748108433316
ANOTHER ROUND!
Check!
1.0934748108433316
ANOTHER ROUND!
Check!
inf
ANOTHER ROUND!
Check!
inf
ANOTHER ROUND!
Check!
inf
ANOTHER ROUND!
Check!
inf
ANOTHER ROUND!
Check!
inf
ANOTHER ROUND!
Check!
inf
ANOTHER ROUND!
Check!
inf
ANOTHER ROUND!
Check!
inf
ANOTHER ROUND!
Check!
inf
ANOTHER ROUND!
Check!
1.062549268691527
ANOTHER ROUND!
Check!
1.062549268691527
ANOTHER ROUND!
Check!
inf
ANOTHER ROUND!
Check!
inf
ANOTHER ROUND!
Check!
inf
ANOTHER ROUND!
Check!
inf
ANOTHER ROUND!
